<a href="https://colab.research.google.com/github/kamath/adaptive-quad/blob/master/Adaptive_Quadrature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!-- # Adaptive Quadrature -->

<!-- Concept - local error is large when function changes a lot (i.e. high $f^{(k)}$)

Instead, we propose an **adaptive grid** where we have smaller grid sizes where f varies a lot, and larger where it doesn't as much

## Math Intuition

We consider an interval $[a, b]$.

$$\text{For an interval }[a, b]\text{ let }h = \frac{1}{2}(b-a)$$

### Simpson's Rule

General form given a function $f$ and start/end points $p$ and $q$:
$$S(f, p, q) = \frac{q-p}{6}[f(p) + 4f(\frac{p+q}{2}) + f(p)]$$

N.B.: we use $p$ and $q$ above

Then,

$$S_1[a, b] = \frac{b-a}{6}[f(a) + 4f(\frac{a+b}{2}) + f(b)]$$

We define this as $S_1$ because it's the first such interval we define over the interval $[a, b]$

### Simpson's Rule Error

$$E_1[a, b] = \frac{-1}{90}h^5f^{(4)}(\xi)\quad\xi\in[a, b]$$

Proof: beyond the scope

### Full Integral (approximation + error)

$$\int_{a}^{b}f\,dx = S_1[a,b] + E_1[a,b]$$

## Mathematical algorithm
Let $m = \frac{a+b}{2}$, the midpoint of a and b

By fundamental theorem of calculus:

$$\int_{a}^{b}f\,dx = \int_{a}^{m}f\,dx + \int_{m}^{b}f\,dx$$

$$= S_1[a, m] + E_1[a, m] + S_1[m, b] + E_1[m, b] $$

$$= S_1[a, m] + S_1[m, b] + E_1[a, m] + E_1[m, b] $$

Let $S_2[a, b] = S_1[a, m] + S_1[m, b]$. Similarly for $E_1$.

Then,

$$E_2 = $$ -->

In [1]:
import numpy as np

In [2]:
simpson = lambda f, a, b: ((b-a) / 6) * (f(a) + 4*f((a+b)/2) + f(b))

def adaptive_quadrature(f, a, b, tol=.001, quad=simpson):
    '''
    Computes adaptive quadrature

    :param f: a function f(x)
    :param a: the start point
    :param b: the end point
    :param tol: the max error desired
    :param quad: the quadrature method (default simpson's rule)
    '''
    c = (a + b) / 2

    s1 = quad(f, a, b)
    s2 = quad(f, a, c) + quad(f, c, b)
    e2 = (1/15) * (s2 - s1)

    if abs(e2) < tol:
        return s2 + e2
    
    left = adaptive_quadrature(f, a, c, tol / 2, quad)
    right = adaptive_quadrature(f, c, b, tol / 2, quad)
    return left + right

def adaptive_step(f, a, b, tol=.001, quad=simpson):
    '''
    Computes adaptive quadrature as a list of steps (same params as above)
    '''
    c = (a + b) / 2

    s1 = quad(f, a, b)
    s2 = quad(f, a, c) + quad(f, c, b)
    e2 = (1/15) * (s2 - s1)
    # print(a, "to", b, "has error", abs(e2))

    if abs(e2) < tol:
        return [[a, c, b, s2 + e2, abs(e2)], [a, (a+c) / 2, c, s2 + e2, abs(e2), "FINAL"], [c, (c+b) / 2, b, s2 + e2, abs(e2), "FINAL"]]
    
    ans = [[a, c, b, s2 + e2, e2 * 2]]
    ans += adaptive_step(f, a, c, tol / 2, quad)
    ans += adaptive_step(f, c, b, tol / 2, quad)
    return ans

def measure(f, i, a, b, tol=.001, quad=simpson):
    '''
    Formats results nicely

    Same params as above, with i(a, b) being the integral function from a to b
    '''
    ans = adaptive_step(f, a, b, tol, quad)
    exp = i(a, b)
    print("")
    for x, mid, y, est, err, *rest in ans:
        print(f"FROM {x} to {y} AREA: {est} ERR: {err}")
        if len(rest):
            print("DONE\n")
    ans = adaptive_quadrature(f, a, b, tol, quad)
    print("FINAL ANSWER:", ans, "FINAL ERROR:", abs(ans - exp))

In [3]:
# Integral of cos from 0 to 5 (sanity check)
measure(np.cos, lambda a, b: np.sin(b) - np.sin(a), 0, 5)


FROM 0 to 5 AREA: -0.9330928848076648 ERR: 0.08345841822451154
FROM 0 to 2.5 AREA: 0.5983780740559801 ERR: -0.0012520029322806714
FROM 0 to 1.25 AREA: 0.9489826487684763 ERR: 5.28401865553881e-05
FROM 0 to 0.625 AREA: 0.9489826487684763 ERR: 5.28401865553881e-05
DONE

FROM 0.625 to 1.25 AREA: 0.9489826487684763 ERR: 5.28401865553881e-05
DONE

FROM 1.25 to 2.5 AREA: -0.3505117474048854 ERR: 1.9516801647286493e-05
FROM 1.25 to 1.875 AREA: -0.3505117474048854 ERR: 1.9516801647286493e-05
DONE

FROM 1.875 to 2.5 AREA: -0.3505117474048854 ERR: 1.9516801647286493e-05
DONE

FROM 2.5 to 5 AREA: -1.5571516214824181 ERR: 0.0032580712439659837
FROM 2.5 to 3.75 AREA: -1.1700310332464805 ERR: 6.514835455904885e-05
FROM 2.5 to 3.125 AREA: -1.1700310332464805 ERR: 6.514835455904885e-05
DONE

FROM 3.125 to 3.75 AREA: -1.1700310332464805 ERR: 6.514835455904885e-05
DONE

FROM 3.75 to 5 AREA: -0.3873621515532294 ERR: 2.1568664484158026e-05
FROM 3.75 to 4.375 AREA: -0.3873621515532294 ERR: 2.1568664484158

In [4]:
from scipy.special import jv # Bessel function
import scipy.integrate as integrate # Test integration

f = lambda x: jv(1, x)
a = 0
b = 5
actual_integral = lambda a, b: integrate.quad(f, a, b)[0]

measure(f, actual_integral, a, b)


FROM 0 to 5 AREA: 1.1708532649342758 ERR: -0.0266430577442188
FROM 0 to 2.5 AREA: 1.0483035865712638 ERR: -0.0012322154152480873
FROM 0 to 1.25 AREA: 0.35409352280846573 ERR: 1.203854537173236e-05
FROM 0 to 0.625 AREA: 0.35409352280846573 ERR: 1.203854537173236e-05
DONE

FROM 0.625 to 1.25 AREA: 0.35409352280846573 ERR: 1.203854537173236e-05
DONE

FROM 1.25 to 2.5 AREA: 0.6942891738318588 ERR: 2.1523807038474155e-05
FROM 1.25 to 1.875 AREA: 0.6942891738318588 ERR: 2.1523807038474155e-05
DONE

FROM 1.875 to 2.5 AREA: 0.6942891738318588 ERR: 2.1523807038474155e-05
DONE

FROM 2.5 to 5 AREA: 0.12924473935330166 ERR: 0.0002019534650103183
FROM 2.5 to 3.75 AREA: 0.12924473935330166 ERR: 0.0002019534650103183
DONE

FROM 3.75 to 5 AREA: 0.12924473935330166 ERR: 0.0002019534650103183
DONE

FINAL ANSWER: 1.1776274359936263 FINAL ERROR: 3.066467928825034e-05


In [5]:
# Sinc function
f = lambda x: np.sin(x) / x
a = 0.01
b = 14.5
actual_integral = lambda a, b: integrate.quad(f, a, b)[0]

measure(f, actual_integral, a, b, tol=.1)


FROM 0.01 to 14.5 AREA: 0.2806166556876185 ERR: -0.423718401100563
FROM 0.01 to 7.255 AREA: 1.5203664283072165 ERR: 0.10028281914091401
FROM 0.01 to 3.6325 AREA: 1.8078521229506088 ERR: 0.00013587159561569232
FROM 0.01 to 1.8212499999999998 AREA: 1.8078521229506088 ERR: 0.00013587159561569232
DONE

FROM 1.8212499999999998 to 3.6325 AREA: 1.8078521229506088 ERR: 0.00013587159561569232
DONE

FROM 3.6325 to 7.255 AREA: -0.33662853681551425 ERR: 0.0001982820580116488
FROM 3.6325 to 5.44375 AREA: -0.33662853681551425 ERR: 0.0001982820580116488
DONE

FROM 5.44375 to 7.255 AREA: -0.33662853681551425 ERR: 0.0001982820580116488
DONE

FROM 7.255 to 14.5 AREA: 0.13677523423400778 ERR: 0.022650203323500755
FROM 7.255 to 10.8775 AREA: 0.13677523423400778 ERR: 0.022650203323500755
DONE

FROM 10.8775 to 14.5 AREA: 0.13677523423400778 ERR: 0.022650203323500755
DONE

FINAL ANSWER: 1.6079988203691025 FINAL ERROR: 0.027275902742820657


In [6]:
# "Wonky" function
f = lambda x: -16 * np.pi / (x+1)**2 * np.sin(4 * np.pi / (x + 1))
a = 0
b = 5
actual_integral = lambda a, b: integrate.quad(f, a, b)[0]

measure(f, actual_integral, a, b, tol=.1)


FROM 0 to 5 AREA: 10.177810191349073 ERR: 0.6563690706804528
FROM 0 to 2.5 AREA: -9.229981425258693 ERR: -2.5761054508645618
FROM 0 to 1.25 AREA: -1.794064779424911 ERR: 1.577940040017748
FROM 0 to 0.625 AREA: 3.7771042005988322 ERR: 0.4916824111560614
FROM 0 to 0.3125 AREA: 7.968373328884408 ERR: 0.0017271857681982656
FROM 0 to 0.15625 AREA: 7.968373328884408 ERR: 0.0017271857681982656
DONE

FROM 0.15625 to 0.3125 AREA: 7.968373328884408 ERR: 0.0017271857681982656
DONE

FROM 0.3125 to 0.625 AREA: -4.438123417829446 ERR: 0.001663868020333285
FROM 0.3125 to 0.46875 AREA: -4.438123417829446 ERR: 0.001663868020333285
DONE

FROM 0.46875 to 0.625 AREA: -4.438123417829446 ERR: 0.001663868020333285
DONE

FROM 0.625 to 1.25 AREA: -2.5803705273719384 ERR: 0.00960567603673829
FROM 0.625 to 0.9375 AREA: -2.5803705273719384 ERR: 0.00960567603673829
DONE

FROM 0.9375 to 1.25 AREA: -2.5803705273719384 ERR: 0.00960567603673829
DONE

FROM 1.25 to 2.5 AREA: 6.667123528510743 ERR: 0.01196669554814124
F

Sources: 

- Wen Shen's Lecture Notes https://www.youtube.com/watch?v=M-AiOqmleaI&ab_channel=wenshenpsu
